In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Hand Detection using Mediapipe Holistic model

In [4]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                 
    results = model.process(image)                
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_styled_landmarks(image, results):

    # left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    key1 = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    key2 = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([key1, key2, lh, rh])


# 4. Setup Folders for Collection

In [8]:
DATA_PATH = os.path.join('dynamic_dataset') 


actions = np.array(['Weather', 'Cloud', 'Rain', 'Thunderstorm', 'Sunny', 'Snow'])

# 30 videos
no_sequences = 40

# 30 frames
sequence_length = 30

In [9]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [10]:
import time

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)

                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            time.sleep(2)
cap.release()
cv2.destroyAllWindows()



# 6. Preprocess Data and Create Labels

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
label_map

{'Weather': 0, 'Cloud': 1, 'Rain': 2, 'Thunderstorm': 3, 'Sunny': 4, 'Snow': 5}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:

X = np.array(sequences)

In [16]:

X.shape

(240, 30, 1662)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:

y_train

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0]])

In [20]:

y_test.shape

(12, 6)

In [21]:

X_train.shape

(228, 30, 1662)


# 7. Build and Train LSTM Neural Network

In [22]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

In [23]:
log_dir = os.path.join('allLogs')
tb_callback = TensorBoard(log_dir=log_dir)

In [24]:

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='sigmoid', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='sigmoid'))
model.add(LSTM(64, return_sequences=False, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [25]:
model.compile(optimizer= 'Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=600, callbacks=[tb_callback], validation_data = (X_test, y_test))

Epoch 1/600
8/8 [==============================] - 3s 99ms/step - loss: 1.9954 - categorical_accuracy: 0.1754 - val_loss: 2.1652 - val_categorical_accuracy: 0.0000e+00
Epoch 2/600
8/8 [==============================] - 0s 50ms/step - loss: 1.8562 - categorical_accuracy: 0.1754 - val_loss: 1.9747 - val_categorical_accuracy: 0.0000e+00
Epoch 3/600
8/8 [==============================] - 0s 42ms/step - loss: 1.8075 - categorical_accuracy: 0.1754 - val_loss: 1.8783 - val_categorical_accuracy: 0.0833
Epoch 4/600
8/8 [==============================] - 0s 43ms/step - loss: 1.7959 - categorical_accuracy: 0.1711 - val_loss: 1.8310 - val_categorical_accuracy: 0.0833
Epoch 5/600
8/8 [==============================] - 0s 47ms/step - loss: 1.7942 - categorical_accuracy: 0.1623 - val_loss: 1.7982 - val_categorical_accuracy: 0.2500
Epoch 6/600
8/8 [==============================] - 0s 47ms/step - loss: 1.7940 - categorical_accuracy: 0.1623 - val_loss: 1.7750 - val_categorical_accuracy: 0.2500
Epoch 7/

8/8 [==============================] - 1s 81ms/step - loss: 1.1549 - categorical_accuracy: 0.3158 - val_loss: 1.1463 - val_categorical_accuracy: 0.3333
Epoch 51/600
8/8 [==============================] - 1s 82ms/step - loss: 1.1538 - categorical_accuracy: 0.3333 - val_loss: 1.1512 - val_categorical_accuracy: 0.3333
Epoch 52/600
8/8 [==============================] - 1s 79ms/step - loss: 1.1521 - categorical_accuracy: 0.2982 - val_loss: 1.1463 - val_categorical_accuracy: 0.2500
Epoch 53/600
8/8 [==============================] - 1s 81ms/step - loss: 1.1480 - categorical_accuracy: 0.3596 - val_loss: 1.1583 - val_categorical_accuracy: 0.0833
Epoch 54/600
8/8 [==============================] - 1s 81ms/step - loss: 1.1474 - categorical_accuracy: 0.3333 - val_loss: 1.1601 - val_categorical_accuracy: 0.1667
Epoch 55/600
8/8 [==============================] - 1s 78ms/step - loss: 1.1469 - categorical_accuracy: 0.3553 - val_loss: 1.1655 - val_categorical_accuracy: 0.0833
Epoch 56/600
8/8 [=====

Epoch 100/600
8/8 [==============================] - 1s 83ms/step - loss: 1.1067 - categorical_accuracy: 0.4123 - val_loss: 1.1311 - val_categorical_accuracy: 0.3333
Epoch 101/600
8/8 [==============================] - 1s 82ms/step - loss: 1.1028 - categorical_accuracy: 0.4211 - val_loss: 1.1328 - val_categorical_accuracy: 0.5833
Epoch 102/600
8/8 [==============================] - 1s 81ms/step - loss: 1.0941 - categorical_accuracy: 0.4211 - val_loss: 1.1263 - val_categorical_accuracy: 0.5833
Epoch 103/600
8/8 [==============================] - 1s 78ms/step - loss: 1.0842 - categorical_accuracy: 0.4518 - val_loss: 1.1124 - val_categorical_accuracy: 0.5833
Epoch 104/600
8/8 [==============================] - 1s 83ms/step - loss: 1.0804 - categorical_accuracy: 0.4123 - val_loss: 1.1225 - val_categorical_accuracy: 0.5000
Epoch 105/600
8/8 [==============================] - 1s 82ms/step - loss: 1.0731 - categorical_accuracy: 0.4474 - val_loss: 1.0583 - val_categorical_accuracy: 0.4167
Epoc

8/8 [==============================] - 1s 80ms/step - loss: 0.8861 - categorical_accuracy: 0.5219 - val_loss: 0.8301 - val_categorical_accuracy: 0.4167
Epoch 150/600
8/8 [==============================] - 1s 76ms/step - loss: 0.8812 - categorical_accuracy: 0.5263 - val_loss: 0.8195 - val_categorical_accuracy: 0.5000
Epoch 151/600
8/8 [==============================] - 1s 82ms/step - loss: 0.8828 - categorical_accuracy: 0.5219 - val_loss: 0.8195 - val_categorical_accuracy: 0.4167
Epoch 152/600
8/8 [==============================] - 1s 79ms/step - loss: 0.8805 - categorical_accuracy: 0.5263 - val_loss: 0.8176 - val_categorical_accuracy: 0.5000
Epoch 153/600
8/8 [==============================] - 1s 78ms/step - loss: 0.8799 - categorical_accuracy: 0.5263 - val_loss: 0.8214 - val_categorical_accuracy: 0.5000
Epoch 154/600
8/8 [==============================] - 1s 83ms/step - loss: 0.8757 - categorical_accuracy: 0.5088 - val_loss: 0.8069 - val_categorical_accuracy: 0.8333
Epoch 155/600
8/8 

8/8 [==============================] - 1s 81ms/step - loss: 0.7726 - categorical_accuracy: 0.6667 - val_loss: 0.7843 - val_categorical_accuracy: 0.5000
Epoch 199/600
8/8 [==============================] - 1s 82ms/step - loss: 0.7638 - categorical_accuracy: 0.6667 - val_loss: 0.7658 - val_categorical_accuracy: 0.5000
Epoch 200/600
8/8 [==============================] - 1s 79ms/step - loss: 0.7563 - categorical_accuracy: 0.6711 - val_loss: 0.7298 - val_categorical_accuracy: 0.5000
Epoch 201/600
8/8 [==============================] - 1s 80ms/step - loss: 0.7506 - categorical_accuracy: 0.6360 - val_loss: 0.7171 - val_categorical_accuracy: 0.5000
Epoch 202/600
8/8 [==============================] - 1s 75ms/step - loss: 0.7437 - categorical_accuracy: 0.6754 - val_loss: 0.7103 - val_categorical_accuracy: 0.5000
Epoch 203/600
8/8 [==============================] - 1s 77ms/step - loss: 0.7379 - categorical_accuracy: 0.6711 - val_loss: 0.7230 - val_categorical_accuracy: 0.5000
Epoch 204/600
8/8 

...

8/8 [==============================] - 1s 79ms/step - loss: 0.1098 - categorical_accuracy: 0.9912 - val_loss: 0.0580 - val_categorical_accuracy: 1.0000
Epoch 395/600
8/8 [==============================] - 1s 80ms/step - loss: 0.1080 - categorical_accuracy: 0.9912 - val_loss: 0.0572 - val_categorical_accuracy: 1.0000
Epoch 396/600
8/8 [==============================] - 1s 79ms/step - loss: 0.1067 - categorical_accuracy: 0.9912 - val_loss: 0.0565 - val_categorical_accuracy: 1.0000
Epoch 397/600
8/8 [==============================] - 1s 78ms/step - loss: 0.1062 - categorical_accuracy: 0.9912 - val_loss: 0.0561 - val_categorical_accuracy: 1.0000
Epoch 398/600
8/8 [==============================] - 1s 83ms/step - loss: 0.1041 - categorical_accuracy: 0.9912 - val_loss: 0.0551 - val_categorical_accuracy: 1.0000
Epoch 399/600
8/8 [==============================] - 1s 81ms/step - loss: 0.1025 - categorical_accuracy: 0.9912 - val_loss: 0.0542 - val_categorical_accuracy: 1.0000
Epoch 400/600
8/8 

8/8 [==============================] - 1s 80ms/step - loss: 0.0750 - categorical_accuracy: 0.9912 - val_loss: 0.0369 - val_categorical_accuracy: 1.0000
Epoch 444/600
8/8 [==============================] - 1s 81ms/step - loss: 0.0745 - categorical_accuracy: 0.9912 - val_loss: 0.0368 - val_categorical_accuracy: 1.0000
Epoch 445/600
8/8 [==============================] - 1s 77ms/step - loss: 0.0741 - categorical_accuracy: 0.9912 - val_loss: 0.0363 - val_categorical_accuracy: 1.0000
Epoch 446/600
8/8 [==============================] - 1s 79ms/step - loss: 0.0738 - categorical_accuracy: 0.9912 - val_loss: 0.0365 - val_categorical_accuracy: 1.0000
Epoch 447/600
8/8 [==============================] - 1s 80ms/step - loss: 0.0731 - categorical_accuracy: 0.9912 - val_loss: 0.0359 - val_categorical_accuracy: 1.0000
Epoch 448/600
8/8 [==============================] - 1s 79ms/step - loss: 0.0727 - categorical_accuracy: 0.9912 - val_loss: 0.0357 - val_categorical_accuracy: 1.0000
Epoch 449/600
8/8 

8/8 [==============================] - 1s 81ms/step - loss: 0.0616 - categorical_accuracy: 0.9912 - val_loss: 0.0273 - val_categorical_accuracy: 1.0000
Epoch 493/600
8/8 [==============================] - 1s 78ms/step - loss: 0.0613 - categorical_accuracy: 0.9912 - val_loss: 0.0270 - val_categorical_accuracy: 1.0000
Epoch 494/600
8/8 [==============================] - 1s 83ms/step - loss: 0.0611 - categorical_accuracy: 0.9912 - val_loss: 0.0264 - val_categorical_accuracy: 1.0000
Epoch 495/600
8/8 [==============================] - 1s 80ms/step - loss: 0.0608 - categorical_accuracy: 0.9912 - val_loss: 0.0262 - val_categorical_accuracy: 1.0000
Epoch 496/600
8/8 [==============================] - 1s 82ms/step - loss: 0.0606 - categorical_accuracy: 0.9912 - val_loss: 0.0270 - val_categorical_accuracy: 1.0000
Epoch 497/600
8/8 [==============================] - 1s 81ms/step - loss: 0.0608 - categorical_accuracy: 0.9912 - val_loss: 0.0272 - val_categorical_accuracy: 1.0000
Epoch 498/600
8/8 

8/8 [==============================] - 1s 76ms/step - loss: 0.0538 - categorical_accuracy: 0.9912 - val_loss: 0.0202 - val_categorical_accuracy: 1.0000
Epoch 542/600
8/8 [==============================] - 1s 82ms/step - loss: 0.0537 - categorical_accuracy: 0.9912 - val_loss: 0.0202 - val_categorical_accuracy: 1.0000
Epoch 543/600
8/8 [==============================] - 1s 83ms/step - loss: 0.0536 - categorical_accuracy: 0.9912 - val_loss: 0.0201 - val_categorical_accuracy: 1.0000
Epoch 544/600
8/8 [==============================] - 1s 69ms/step - loss: 0.0535 - categorical_accuracy: 0.9912 - val_loss: 0.0200 - val_categorical_accuracy: 1.0000
Epoch 545/600
8/8 [==============================] - 1s 77ms/step - loss: 0.0534 - categorical_accuracy: 0.9912 - val_loss: 0.0198 - val_categorical_accuracy: 1.0000
Epoch 546/600
8/8 [==============================] - 1s 81ms/step - loss: 0.0532 - categorical_accuracy: 0.9912 - val_loss: 0.0197 - val_categorical_accuracy: 1.0000
Epoch 547/600
8/8 

8/8 [==============================] - 1s 83ms/step - loss: 0.0661 - categorical_accuracy: 0.9912 - val_loss: 0.0532 - val_categorical_accuracy: 1.0000
Epoch 591/600
8/8 [==============================] - 1s 80ms/step - loss: 0.0626 - categorical_accuracy: 0.9912 - val_loss: 0.0503 - val_categorical_accuracy: 1.0000
Epoch 592/600
8/8 [==============================] - 1s 79ms/step - loss: 0.0604 - categorical_accuracy: 0.9912 - val_loss: 0.0449 - val_categorical_accuracy: 1.0000
Epoch 593/600
8/8 [==============================] - 1s 80ms/step - loss: 0.0623 - categorical_accuracy: 0.9912 - val_loss: 0.0487 - val_categorical_accuracy: 1.0000
Epoch 594/600
8/8 [==============================] - 1s 78ms/step - loss: 0.0604 - categorical_accuracy: 0.9912 - val_loss: 0.0405 - val_categorical_accuracy: 1.0000
Epoch 595/600
8/8 [==============================] - 1s 79ms/step - loss: 0.0574 - categorical_accuracy: 0.9912 - val_loss: 0.0321 - val_categorical_accuracy: 1.0000
Epoch 596/600
8/8 

# 8. Make Predictions

In [27]:

res = model.predict(X_test)

In [28]:

actions[np.argmax(res[2])]

'Weather'

In [29]:
actions[np.argmax(y_test[2])]

'Weather'

In [30]:
from tensorflow import keras

In [31]:
 model.save('rec_1.h5')

In [32]:
modelF= keras.models.load_model('rec_1.h5')

In [33]:

model.load_weights('rec_1.h5')

# 10. Evaluation and Accuracy

In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [35]:
yhtst = modelF.predict(X_test)
yhtrn = modelF.predict(X_train)

In [36]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhtst = np.argmax(yhtst, axis=1).tolist()

In [37]:
ytrain = np.argmax(y_train, axis=1).tolist()
yhtrn = np.argmax(yhtrn, axis=1).tolist()

In [38]:
multilabel_confusion_matrix(ytrue, yhat)

NameError: name 'yhat' is not defined

In [39]:
accuracy_score(ytrue, yhtst)

1.0

In [40]:
accuracy_score(ytrain, yhtrn)

0.9912280701754386

# 11. Test in Real Time

In [42]:
colors = [(245,117,16), (117,245,16), (16,117,245),(16,17,245),(16,117,24),(17,25,160)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [46]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.93

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = modelF.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Recognition Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediap

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thunderstorm
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Weather
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.Solution

Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<cl

Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sunny
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Rain
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Rain
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Rain
<class 'mediapipe.python.solution_base.SolutionOutpu

Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Snow
<cl